
# Glue Studio Notebook
You are now running a **Glue Studio** notebook; before you can start using your notebook you *must* start an interactive session.

## Available Magics
|          Magic              |   Type       |                                                                        Description                                                                        |
|-----------------------------|--------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|
| %%configure                 |  Dictionary  |  A json-formatted dictionary consisting of all configuration parameters for a session. Each parameter can be specified here or through individual magics. |
| %profile                    |  String      |  Specify a profile in your aws configuration to use as the credentials provider.                                                                          |
| %iam_role                   |  String      |  Specify an IAM role to execute your session with.                                                                                                        |
| %region                     |  String      |  Specify the AWS region in which to initialize a session                                                                                                  |
| %session_id                 |  String      |  Returns the session ID for the running session.                                                                                                          |
| %connections                |  List        |  Specify a comma separated list of connections to use in the session.                                                                                     |
| %additional_python_modules  |  List        |  Comma separated list of pip packages, s3 paths or private pip arguments.                                                                                 |
| %extra_py_files             |  List        |  Comma separated list of additional Python files from S3.                                                                                                 |
| %extra_jars                 |  List        |  Comma separated list of additional Jars to include in the cluster.                                                                                       |
| %number_of_workers          |  Integer     |  The number of workers of a defined worker_type that are allocated when a job runs. worker_type must be set too.                                          |
| %worker_type                |  String      |  Standard, G.1X, *or* G.2X. number_of_workers must be set too. Default is G.1X                                                                            |
| %glue_version               |  String      |  The version of Glue to be used by this session. Currently, the only valid options are 2.0 and 3.0 (eg: %glue_version 2.0)                                |
| %security_configuration     |  String      |  Define a security configuration to be used with this session.                                                                                            |
| %sql                        |  String      |  Run SQL code. All lines after the initial %%sql magic will be passed as part of the SQL code.                                                            |
| %streaming                  |  String      |  Changes the session type to Glue Streaming.                                                                                                              |
| %etl                        |  String      |   Changes the session type to Glue ETL.                                                                                                                   |
| %status                     |              |  Returns the status of the current Glue session including its duration, configuration and executing user / role.                                          |
| %stop_session               |              |  Stops the current session.                                                                                                                               |
| %list_sessions              |              |  Lists all currently running sessions by name and ID.                                                                                                     |

In [5]:
%region ap-northeast-2 
%number_of_workers 2
%worker_type G.1X
%glue_version 3.0
%extra_py_files s3://poc-glue/script/batch/daliy/layer.zip,s3://poc-glue/script/daliy/libs.zip

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
It looks like there is a newer version of the kernel available. The latest version is 0.35 and you have 0.30 installed.
Please run `pip install --upgrade aws-glue-sessions` to upgrade your kernel
Previous region: ap-northeast-2
Setting new region to: ap-northeast-2
Reauthenticating Glue client with new region: ap-northeast-2
IAM role has been set to arn:aws:iam::711159021539:role/ops-aico-nurikids-glue-base-role. Reauthenticating.
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::711159021539:role/ops-aico-nurikids-glue-base-role
Attempting to use existing AssumeRole session credentials.
Authentication done.
Region is set to: ap-northeas

In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 2
Session ID: 9035e80e-069d-4e02-80a8-fa2e3a3fa7a1
Applying the following default arguments:
--glue_kernel_version 0.30
--enable-glue-datacatalog true
Waiting for session 9035e80e-069d-4e02-80a8-fa2e3a3fa7a1 to get into ready status...
Session 9035e80e-069d-4e02-80a8-fa2e3a3fa7a1 has been created




# Read parquet data from s3

In [ ]:
bucket = 's3://버킷명/'
s3_path = 'poc/daily/'

read_path = bucket + s3_path 

# make dataframe
df_data = spark.read.parquet(read_path)
df_data = df_data.distinct() 

# Read data from dynamodb

In [6]:
def get_df_from_dynamodb(pGlueContext:GlueContext, pTableNm:str):
    dynamic_frame = pGlueContext.create_dynamic_frame.from_options(
        connection_type="dynamodb",
        connection_options={
            "dynamodb.input.tableName": pTableNm,
            "dynamodb.throughput.read.percent": "0.1",
            "dynamodb.splits": "1"
        }
    )
    df_dynamodb = dynamic_frame.toDF()

    # df_dynamodb.cache()
    # df_dynamodb.createOrReplaceTempView(pTableNm)
    return df_dynamodb

In [7]:
df_data = get_df_from_dynamodb(glueContext, '다이나모디비명')

In [ ]:
from layer.common import * # s3://poc-glue/script/batch/daliy/layer.zip
from libs.preprocessing import * # s3://poc-glue/script/daliy/libs.zip